In [ ]:
#| hide
from uom_project import poisson_solvers

import numpy as np

# uom-project

> Repository containing code and and other materials for my undergraduate project on numerical solutions to the Navier-Stokes equations at the University of Manchester.

## Install

```sh
pip install uom_project
```

## How to use

### Poisson solvers

#### Gauss-Seidel

In [ ]:
N = 20
nx = ny = N + 1
h = 1 / N

# SOR parameter
r = 2 / (1 + np.pi / N) # optimal value

# Initialize grid and the value of the vorticity `w` on it
x_grid, y_grid = np.meshgrid(
    np.linspace(0, 1, nx), np.linspace(0, 1, ny), indexing="ij"
)
w = 2 * np.pi ** 2 * np.sin(np.pi * x_grid) * np.sin(np.pi * y_grid)

# Compute the exact solution
exact_solution = np.sin(np.pi * x_grid) * np.sin(np.pi * y_grid)

In [ ]:
psi = poisson_solvers.poisson_gauss_seidel_with_sor_solver(w, verbose=False,)[0]

# Check absolute error is small
np.abs(psi - exact_solution).max()

0.0020587065214789924

In [ ]:
psi = poisson_solvers.poisson_non_iterative_solver(w)

# Check absolute error is small
np.abs(psi - exact_solution).max()

0.002058706764533902

In [ ]:
options = { "line_search": None, "jac_options": { "reduction_method": "svd" } }
psi, solution = poisson_solvers.newton_alternative(w, method="broyden2", options=options)

# Check absolute error is small
np.abs(psi - exact_solution).max()

0.002058706764533458

#### SciPy solvers